In [1]:
import pandas as pd

In [2]:
data_xls = pd.ExcelFile('input/supply_chain_data.xlsx')
data_df_map = {sheet_name: data_xls.parse(sheet_name) for sheet_name in data_xls.sheet_names}

In [3]:
order_list = data_df_map['OrderList'].copy()
freight_rates = data_df_map['FreightRates'].copy()
wh_costs = data_df_map['WhCosts'].copy()
wh_capacities = data_df_map['WhCapacities'].copy()
products_per_plant = data_df_map['ProductsPerPlant'].copy()
vmi_customers = data_df_map['VmiCustomers'].copy()
plant_ports = data_df_map['PlantPorts'].copy()

# Cleaning the data

## Clean Abnormal Data

In [4]:
# Deal with anomaly plant code in sheet products_per_plant
products_per_plant.loc[products_per_plant["Plant Code"] == "CND9", "Plant Code"] = "PLANT09"

# Clean whitespace in some columns
def strip_whitespace(string):
    return string.strip()
order_list["Service Level"] = order_list["Service Level"].apply(strip_whitespace)
freight_rates['svc_cd'] = freight_rates['svc_cd'].apply(strip_whitespace)
freight_rates['mode_dsc'] = freight_rates['mode_dsc'].apply(strip_whitespace)

# Remove white space in column names
wh_capacities.columns = wh_capacities.columns.str.strip()

# drop useless columns
freight_rates.drop(columns=["Carrier type"], inplace=True)

## Change Every Object into an Index mapping

In [5]:
# Order List
## Order id
orders = order_list["Order ID"].unique().tolist()
orders.sort()
order_dic = {}
for i in range(len(orders)):
    order_dic[orders[i]] = i + 1

## Warehouse id
warehouses_1 = wh_costs["WH"].unique().tolist()
warehouses_2 = wh_capacities["Plant ID"].unique().tolist()
warehouses_3 = products_per_plant["Plant Code"].unique().tolist()
warehouses_4 = vmi_customers["Plant Code"].unique().tolist()
warehouses_5 = plant_ports["Plant Code"].unique().tolist()
warehouses = list(set(warehouses_1 + warehouses_2 + warehouses_3 + warehouses_4 + warehouses_5))
warehouses.sort()
warehouse_dic = {}
for i in range(len(warehouses)):
    warehouse_dic[warehouses[i]] = i + 1

## Product id
products_1 = order_list["Product ID"].unique().tolist()
products_2 = products_per_plant["Product ID"].unique().tolist()
products = list(set(products_1 + products_2))
products.sort()
product_dic = {}
for i in range(len(products)):
    product_dic[products[i]] = i + 1

## Wharehouse Port id (Original Port)
warehouse_ports_1 = order_list["Origin Port"].unique().tolist()
warehouse_ports_2 = freight_rates["orig_port_cd"].unique().tolist()
warehouse_ports_3 = plant_ports["Port"].unique().tolist()
warehouse_ports = list(set(warehouse_ports_1 + warehouse_ports_2 + warehouse_ports_3))
warehouse_ports.sort()
warehouse_port_dic = {}
for i in range(len(warehouse_ports)):
    warehouse_port_dic[warehouse_ports[i]] = i + 1

## Customer Port id (Destination Port)
customer_ports_1 = order_list["Destination Port"].unique().tolist()
customer_ports_2 = freight_rates["dest_port_cd"].unique().tolist()
customer_ports = list(set(customer_ports_1 + customer_ports_2))
customer_ports.sort()
customer_port_dic = {}
for i in range(len(customer_ports)):
    customer_port_dic[customer_ports[i]] = i + 1

## Customer id
customers_1 = order_list["Customer"].unique().tolist()
customers_2 = vmi_customers["Customers"].unique().tolist()
customers = list(set(customers_1 + customers_2))
customers.sort()
customer_dic = {}
for i in range(len(customers)):
    customer_dic[customers[i]] = i + 1

## Carrier id
carriers = freight_rates["Carrier"].unique().tolist()
carriers.sort()
carrier_dic = {}
for i in range(len(carriers)):
    carrier_dic[carriers[i]] = i + 1

## Service Level id
service_levels = order_list["Service Level"].unique().tolist()
service_levels.sort()
service_level_dic = {}
for i in range(len(service_levels)):
    if service_levels[i] ==  'CRF':
        service_level_dic[service_levels[i]] = 0
    else:
        service_level_dic[service_levels[i]] = 1
        
## Transportation Mode id
transportation_modes = freight_rates["mode_dsc"].unique().tolist()
transportation_modes.sort()
transportation_mode_dic = {}
for i in range(len(transportation_modes)):
    if transportation_modes[i] == 'AIR':
        transportation_mode_dic[transportation_modes[i]] = 0
    else:
        transportation_mode_dic[transportation_modes[i]] = 1

## Clean each sheet

### 1 order_list

In [6]:
# transfer the certain columns into index
order_list['Order ID'] = order_list['Order ID'].map(order_dic)
order_list['Origin Port'] = order_list['Origin Port'].map(warehouse_port_dic)
order_list['Carrier'] = order_list['Carrier'].map(carrier_dic)
order_list['Service Level'] = order_list['Service Level'].map(service_level_dic)
order_list['Customer'] = order_list['Customer'].map(customer_dic)
order_list['Product ID'] = order_list['Product ID'].map(product_dic)
order_list['Plant Code'] = order_list['Plant Code'].map(warehouse_dic)
order_list['Destination Port'] = order_list['Destination Port'].map(customer_port_dic)
order_list['Maximum delivery time'] = order_list['TPT'] + order_list['Ship ahead day count'] + order_list['Ship Late Day count']
order_list.drop(columns=["Order Date", "TPT", "Ship ahead day count", "Ship Late Day count"], inplace=True)
order_list

,Order ID,Origin Port,Carrier,Service Level,Customer,Product ID,Plant Code,Destination Port,Unit quantity,Weight,Maximum delivery time
0,5276,9,NaN,0,37,1481,16,1,808,14.300000,4
1,691,9,NaN,0,37,1481,16,1,3188,87.940000,4
2,68,9,NaN,0,37,1481,16,1,2331,61.200000,4
3,7361,9,NaN,0,37,1481,16,1,847,16.160000,4
4,7363,9,NaN,0,37,1481,16,1,2163,52.340000,4
...,...,...,...,...,...,...,...,...,...,...,...
9210,5558,4,2.0,1,10,609,3,1,339,2.354118,6
9211,6031,4,2.0,1,10,609,3,1,339,2.354118,6
9212,6202,4,2.0,1,10,609,3,1,245,0.294265,6
9213,220,4,2.0,1,10,623,3,1,278,2.480000,6


### 2 freight_rates

In [7]:
# Add "Area" and "Index" columns to freight_rates
freight_rates['Area'] = freight_rates.groupby(['orig_port_cd', 'dest_port_cd', 'Carrier', 'svc_cd']).cumcount() + 1

# transfer the certain columns into index
freight_rates["Carrier"] = freight_rates["Carrier"].map(carrier_dic)
freight_rates["orig_port_cd"] = freight_rates["orig_port_cd"].map(warehouse_port_dic)
freight_rates["dest_port_cd"] = freight_rates["dest_port_cd"].map(customer_port_dic)
freight_rates["svc_cd"] = freight_rates["svc_cd"].map(service_level_dic)
freight_rates["mode_dsc"] = freight_rates["mode_dsc"].map(transportation_mode_dic)
freight_rates

,Carrier,orig_port_cd,dest_port_cd,minm_wgh_qty,max_wgh_qty,svc_cd,minimum cost,rate,mode_dsc,tpt_day_cnt,Area
0,6,8,1,250.0,499.99,1,43.2272,0.7132,0,2,1
1,6,8,1,65.0,69.99,1,43.2272,0.7512,0,2,2
2,6,8,1,60.0,64.99,1,43.2272,0.7892,0,2,3
3,6,8,1,50.0,54.99,1,43.2272,0.8272,0,2,4
4,6,8,1,35.0,39.99,1,43.2272,1.0552,0,2,5
...,...,...,...,...,...,...,...,...,...,...,...
1535,8,3,1,0.0,99.99,1,29.3392,0.4116,0,0,16
1536,8,3,1,500.0,1999.99,1,29.3392,0.3112,0,0,17
1537,8,3,1,250.0,499.99,1,29.3392,0.4116,0,0,18
1538,8,3,1,2000.0,99999.99,1,29.3392,0.3112,0,0,19


### 3 wh_costs

In [8]:
# transfer the warehouse name into index
wh_costs["WH"] = wh_costs["WH"].map(warehouse_dic)
wh_costs

,WH,Cost/unit
0,15,1.415063
1,17,0.428947
2,18,2.036254
3,5,0.488144
4,2,0.477504
5,1,0.566976
6,6,0.554088
7,10,0.493582
8,7,0.371424
9,14,0.634330


### 4 wh_capacities

In [9]:
# transfer the warehouse name into index
wh_capacities["Plant ID"] = wh_capacities["Plant ID"].map(warehouse_dic)
wh_capacities

,Plant ID,Daily Capacity
0,15,550
1,17,400
2,18,5550
3,5,19250
4,2,6900
5,1,53500
6,6,2450
7,10,5900
8,7,13250
9,14,27450


### 5 products_per_plant

In [10]:
# transfer the warehouse name into index
products_per_plant["Plant Code"] = products_per_plant["Plant Code"].map(warehouse_dic)
# transfer the product id into index
products_per_plant["Product ID"] = products_per_plant["Product ID"].map(product_dic)
products_per_plant

,Plant Code,Product ID
0,15,1323
1,17,173
2,17,174
3,17,333
4,17,368
...,...,...
2031,4,1464
2032,4,1465
2033,4,1466
2034,4,1467


### 6 vmi_customers

In [11]:
# transfer the warehouse name into index
vmi_customers["Plant Code"] = vmi_customers["Plant Code"].map(warehouse_dic)
# transfer the customer name into index
vmi_customers["Customers"] = vmi_customers["Customers"].map(customer_dic)
vmi_customers

,Plant Code,Customers
0,2,11
1,2,8
2,2,17
3,2,10
4,2,23
5,2,33
6,2,21
7,6,6
8,6,33
9,10,8


### 7 plant_ports

In [12]:
# transfer the warehouse name into index
plant_ports["Plant Code"] = plant_ports["Plant Code"].map(warehouse_dic)
# transfer the warehouse port name into index
plant_ports["Port"] = plant_ports["Port"].map(warehouse_port_dic)
vmi_customers

,Plant Code,Customers
0,2,11
1,2,8
2,2,17
3,2,10
4,2,23
5,2,33
6,2,21
7,6,6
8,6,33
9,10,8


# Oputupt Files

In [13]:
# output csv files
order_list.to_csv('input/order_list.csv', index=False)
freight_rates.to_csv('input/freight_rates.csv', index=False)
wh_costs.to_csv('input/wh_costs.csv', index=False)
wh_capacities.to_csv('input/wh_capacities.csv', index=False)
products_per_plant.to_csv('input/products_per_plant.csv', index=False)
vmi_customers.to_csv('input/vmi_customers.csv', index=False)
plant_ports.to_csv('input/plant_ports.csv', index=False)